In [2]:
import os

from python_speech_features import mfcc
from python_speech_features import delta
from python_speech_features import logfbank
import scipy.io.wavfile as wav
from random import randint

In [163]:
no_of_features = 20
no_of_frames = 25

no_of_columns = 3 * no_of_features * no_of_frames

def get_feature_vectors(dataset_type):
    
    #set parameters for training and testing
    if (dataset_type == "train"):
        directory = os.path.join(os.getcwd(), 'data_thuyg20_sre/enroll')
#         no_of_frames = 40
        no_of_cycles = 80
    elif (dataset_type == "test"):    
        directory = os.path.join(os.getcwd(), 'data_thuyg20_sre/test')
#         no_of_frames = 40
        no_of_cycles = 1
    
    dataset = numpy.empty([0, no_of_columns + 1])
    stats_dataset = numpy.empty([0, 3 * no_of_features])
    
    for file in os.listdir(directory):
        
        # filter speakers
        names = ['F101', 'F102', 'F103', 'F104', 'F105', 'M101', 'M102', 'M103']

        if any(name in file for name in names):
            
            # extract mfcc vectors
            (rate,sig) = wav.read(os.path.join(directory, file))
            mfcc_feat = mfcc(sig,rate)
            d_mfcc_feat = delta(mfcc_feat, 2)
            dd_mfcc_feat = delta(d_mfcc_feat, 2)
            
            for x in range(0, no_of_cycles):
                
                random_int = randint(1, 300)

                fbank_feat = logfbank(sig,rate)
                mfcc_vectors = mfcc_feat[random_int:random_int+no_of_frames, :no_of_features]
                dmfcc_vectors = d_mfcc_feat[random_int:random_int+no_of_frames, :no_of_features]
#                 ddmfcc_vectors = dd_mfcc_feat[random_int:random_int+no_of_frames, :no_of_features]
                fbank_vectors = fbank_feat[random_int:random_int+no_of_frames, :no_of_features]
                
                feature_vectors = numpy.hstack((mfcc_vectors, dmfcc_vectors, fbank_vectors))
                
                stats_dataset = numpy.concatenate((stats_dataset, feature_vectors), axis=0)
                
                feature_vectors = feature_vectors.flatten().reshape((1, -1))

                # get speaker index from filename
                speaker_index = file.split("_")[0]
                if speaker_index[0] == 'M':
                    speaker_index = 5 + int(speaker_index[3:])
                else:
                    speaker_index = int(speaker_index[3:])

                #append speaker index to feature vectors
                np_speaker_index = numpy.array([speaker_index])
                temp = numpy.tile(np_speaker_index[numpy.newaxis,:], (feature_vectors.shape[0],1))
                concatenated_feature_vector = numpy.concatenate((feature_vectors,temp), axis=1)

#                 print(concatenated_feature_vector.shape)
                # append file's data to dataset
                dataset = numpy.concatenate((dataset, concatenated_feature_vector), axis=0)

    
    mean = stats_dataset.mean(0, keepdims=True)
    std_deviation = numpy.std(stats_dataset, axis=0, keepdims=True)
    
    if (dataset_type == "train"):
        return dataset, mean, std_deviation
    elif (dataset_type == "test"):    
        return dataset
    

In [117]:
# def get_feature_vectors(dataset_type):
    
#     #set parameters for training and testing
#     if (dataset_type == "train"):
#         directory = os.path.join(os.getcwd(), 'data_thuyg20_sre/enroll')
#         no_of_frames = 40
#     elif (dataset_type == "test"):    
#         directory = os.path.join(os.getcwd(), 'data_thuyg20_sre/test')
#         no_of_frames = 40
    
#     dataset = numpy.empty([0, 61])
    
#     for file in os.listdir(directory):
        
#         # filter speakers
#         names = ['F101', 'F102', 'F103', 'F104', 'F105', 'M101', 'M102', 'M103', 'M104']

#         if any(name in file for name in names):
            
#             # extract mfcc vectors
#             (rate,sig) = wav.read(os.path.join(directory, file))
#             mfcc_feat = mfcc(sig,rate)
#             d_mfcc_feat = delta(mfcc_feat, 2)
#             dd_mfcc_feat = delta(d_mfcc_feat, 2)
            
# #             fbank_feat = logfbank(sig,rate)
#             mfcc_vectors = mfcc_feat[11:11+no_of_frames,:]
#             dmfcc_vectors = d_mfcc_feat[11:11+no_of_frames,:]
#             ddmfcc_vectors = dd_mfcc_feat[11:11+no_of_frames,:]
            
#             feature_vectors = numpy.hstack((mfcc_vectors, dmfcc_vectors, ddmfcc_vectors))
# #             print(feature_vectors.shape)
            
#             # get speaker index from filename
#             speaker_index = file.split("_")[0]
#             if speaker_index[0] == 'M':
#                 speaker_index = 5 + int(speaker_index[3:])
#             else:
#                 speaker_index = int(speaker_index[3:])

#             #append speaker index to feature vectors
#             np_speaker_index = numpy.array([speaker_index])
#             temp = numpy.tile(np_speaker_index[numpy.newaxis,:], (feature_vectors.shape[0],1))
#             concatenated_feature_vector = numpy.concatenate((feature_vectors,temp), axis=1)
            
#             # append file's data to dataset
#             dataset = numpy.concatenate((dataset, concatenated_feature_vector), axis=0)
            

#     return dataset


In [118]:
from keras.models import Sequential
import numpy as numpy 

In [164]:
# from numpy import genfromtxt
my_data, mean, std_deviation = get_feature_vectors("train")


In [165]:
# print(my_data)
print(my_data.shape)

(640, 1501)


In [166]:
Y = numpy.copy(my_data[:, no_of_columns:])
print(Y.shape)

(640, 1)


In [167]:
X = numpy.copy(my_data[:, :no_of_columns])
print(X.shape)

mean = numpy.tile(mean, no_of_frames)
print(mean.shape)

std_deviation = numpy.tile(std_deviation, no_of_frames)
print(std_deviation.shape)

normalized_X = (X - mean) / std_deviation
print(normalized_X.shape)


(640, 1500)
(1, 1500)
(1, 1500)
(640, 1500)


In [123]:
from keras import utils as np_utils

one_hot_labels = np_utils.to_categorical(Y, num_classes=10)
print(one_hot_labels.shape)

(640, 10)


In [247]:
from keras.layers import Dense, Activation, Dropout
from keras.optimizers import SGD

# model = Sequential()
# model.add(Dense(32, activation='relu', input_dim=12))
# model.add(Dense(10, activation='softmax'))

# model.compile(optimizer='rmsprop',
#               loss='categorical_crossentropy',
#               metrics=['accuracy'])

# model.fit(normalized_X, one_hot_labels, epochs=10, batch_size=32)


# MultiLayer Perceptron
model = Sequential()

model.add(Dense(1000, activation='tanh', input_dim=no_of_columns))
# model.add(Dropout(0.5))
model.add(Dense(1000, activation='tanh', use_bias=True))
model.add(Dense(900, activation='tanh', use_bias=True))
# # model.add(Dropout(0.5))
# model.add(Dense(800, activation='tanh', use_bias=True))
model.add(Dense(600, activation='tanh', use_bias=True))
model.add(Dense(400, activation='tanh', use_bias=True))
# model.add(Dense(400, activation='tanh', use_bias=True))
# model.add(Dense(4000, activation='relu'))
# model.add(Dropout(0.5))
# model.add(Dense(4000, activation='relu'))
# model.add(Dropout(0.5))
# model.add(Dense(4000, activation='relu'))
# model.add(Dropout(0.5))
model.add(Dense(10, activation='softmax'))

sgd = SGD(lr=0.005, decay=1e-6, momentum=0.9, nesterov=True)
model.compile(loss='categorical_crossentropy',
              optimizer=sgd,
              metrics=['accuracy'])

model.fit(normalized_X, one_hot_labels, epochs=10, batch_size=32)
# score = model.evaluate(x_test, y_test, batch_size=128)

Epoch 1/10
640/640 [==============================] - 2s 3ms/step - loss: 1.5453 - acc: 0.4969
Epoch 2/10
640/640 [==============================] - 2s 3ms/step - loss: 0.3489 - acc: 0.9141
Epoch 3/10
640/640 [==============================] - 2s 3ms/step - loss: 0.1379 - acc: 0.9719
Epoch 4/10
640/640 [==============================] - 2s 3ms/step - loss: 0.0778 - acc: 0.9797
Epoch 5/10
640/640 [==============================] - 2s 4ms/step - loss: 0.0665 - acc: 0.9797
Epoch 6/10
640/640 [==============================] - 3s 5ms/step - loss: 0.0491 - acc: 0.9844
Epoch 7/10
640/640 [==============================] - 3s 5ms/step - loss: 0.0445 - acc: 0.9859
Epoch 8/10
640/640 [==============================] - 2s 3ms/step - loss: 0.0551 - acc: 0.9797
Epoch 9/10
640/640 [==============================] - 2s 3ms/step - loss: 0.0400 - acc: 0.9859
Epoch 10/10
640/640 [==============================] - 2s 3ms/step - loss: 0.0373 - acc: 0.9859


In [256]:
test_model = get_feature_vectors("test")
print(test_model.shape)

test_X = numpy.copy(test_model[:, :no_of_columns])
print(test_X.shape)

normalized_test_X = (test_X - mean) / std_deviation
print(normalized_test_X.shape)

test_Y = numpy.copy(test_model[:, no_of_columns:])
print(test_Y.shape)
test_labels = np_utils.to_categorical(test_Y, num_classes=10)


(70, 1501)
(70, 1500)
(70, 1500)
(70, 1)


In [257]:
print(model.test_on_batch(test_X, test_labels, sample_weight=None))
print(model.metrics_names)
predictions = model.predict(test_X).argmax(axis=1)
print(predictions)

[array(2.2090818881988525, dtype=float32), array(0.4571428596973419, dtype=float32)]
['loss', 'acc']
[1 1 6 5 1 4 2 1 5 3 1 1 3 6 3 2 5 1 2 2 5 6 2 7 1 3 1 3 2 1 3 5 1 1 1 6 1
 3 7 3 1 1 1 2 8 1 1 1 1 1 1 3 1 1 8 1 1 3 2 7 8 5 7 1 1 5 5 5 8 1]


In [184]:
# b = [sum(predictions[current: current+40]) for current in range(0, len(predictions), 40)]
# predicted_Y = []
# for row in b:
#     predicted_Y.append(row.argmax(axis=0))
    
# # print(predicted_Y)
# # print(test_Y[::40].T)

# # for t, p in zip(test_Y[::40].T[0], predicted_Y):
# #     print (int(t), p)

# diff = predicted_Y - test_Y[::40].T[0]

# print(len(predicted_Y))
# print(sum(x == 0 for x in diff))

In [185]:
# print(test_Y.T)

for t, p in zip(test_Y.T[0], predictions):
    print (int(t), p)

diff = predictions - test_Y.T[0]

print(len(predictions))
print(sum(x == 0 for x in diff))

2 6
1 3
6 3
4 1
1 3
4 3
1 1
5 3
5 5
1 1
1 3
3 1
6 6
6 5
3 3
5 3
3 3
5 1
2 3
4 3
5 5
6 7
2 2
7 7
4 7
3 3
1 1
7 7
1 1
4 3
1 1
5 5
1 1
4 4
4 2
6 6
1 3
1 1
5 3
3 6
1 3
1 1
3 3
5 3
6 2
5 3
4 5
4 5
1 3
5 1
4 1
7 3
1 2
7 3
6 7
3 3
8 1
3 1
2 1
7 5
3 1
3 1
7 7
1 3
1 1
5 5
3 3
3 5
8 1
5 1
70
26
